<a href="https://colab.research.google.com/github/mikaelvesavuori/cloud-functions-object-detection-inference/blob/master/Google_Cloud_Functions_TensorFlow_1_15_for_Object_Detection_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Cloud Functions: TensorFlow 1.15 for Object Detection inference

This sample function is a fairly minimal implementation of how you can do object detection inference in a Google Cloud Function. It will not assist in any of the training steps, for which I recommend Colaboratory or some other dedicated environment.

**Note**:
- This notebook should be able to "emulate" the experience of the function, but you will likely need to do a bit of copy-paste back-and-forth to ensure your real Cloud Function works as intended, as you make any changes of your own.
- There are very small differences between using this in Colab and in a real function. For example: In Colab you'll need to run the first two cells to prep the environments. Those two cells should not be part of your Cloud Function source code!
- This implementation is not optimized for container reuse and performance
- Images will be loaded from source as a file, NOT as Base64 (in which case you probably need to modify the below code, as well as use a model that takes in `encoded_image_string_tensor`)

## Need to (re)train an object detection model?

I have a notebook at https://colab.research.google.com/drive/1FaMfcgskz84VWPb3fAAPQczFTND-zbBl that might help you.

## Prerequisites

- Google Cloud Platform account
- Ready-to-go model (frozen inference graph; `image_tensor` input assumed)
- `label_map.pbtxt` with your labels

## Function specifications

- Python 3.7 runtime (Guess it works fine in Python 3.8 as well)
- 2 GB RAM should give a bit of a boost to the inference speed
- Set the following content in your `requirements.txt` file.

```
tensorflow==1.15.2
google-cloud-storage==1.28.1
Pillow==7.1.2
```

## References

These are some of the resources I've learned from and re-adapted below:

- https://cloud.google.com/blog/products/ai-machine-learning/how-to-serve-deep-learning-models-using-tensorflow-2-0-with-cloud-functions
- https://colab.research.google.com/github/tensorflow/models/blob/master/research/object_detection/object_detection_tutorial.ipynb

Thanks also to various answers on Stack Overflow that I could not find in my history... :)

## Colab-only prep steps

In [0]:
# Only run this prep step if "emulating" inside of Colaboratory!
# In the real function, you will use the download_blob() function to fetch assets

%cd /content/

! curl -O -L https://materials

In [0]:
# Only run this prep step if "emulating" inside of Colaboratory!
# In the real function, you will set the TensorFlow requirement to be "1.15.2"

# Force TF v1 since this is the one that actually works for object detection (as of April 2020)
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


## Cloud Functions + Colab

Look out for any comments on changes required between CF and Colab!

In [0]:
import os
import json
from PIL import Image
import numpy as np
import tensorflow as tf

# Use "/content/" when in Colaboratory
# Use "/tmp/" when in actual Cloud Function environment
BUCKET_NAME = 'my-ml-models'
PATH_TO_MODEL = '/content/frozen_inference_graph_image_tensor.pb'
PATH_TO_LABELS = '/content/label_map.pbtxt'
PATH_TO_IMAGE = '/content/test.jpeg'

# Have a local array/list copy of classes in "label_map.pbtxt", so we can easily pull and match classes and avoid additional file operations.
# This MUST map to the same pattern and ordering as the "label_map.pbtxt" file!
model_classes = [
  'some-class-1',
  'some-class-2',
  'some-class-3',
  'some-class-4'
]

In [0]:
def download_blob(bucket_name, source_blob_name, destination_file_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(source_blob_name)

    blob.download_to_filename(destination_file_name)

    print('Blob {} downloaded to {}.'.format(
        source_blob_name,
        destination_file_name))


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


def handler(request):
    image_url = ''

    request_json = request.get_json(silent=True)
    request_args = request.args

    if request_json and 'image_url' in request_json:
        image_url = request_json['image_url']
    elif request_args and 'image_url' in request_args:
        request_json = request_args['image_url']

    print('image_url:', image_url)

    """
    The function will need to grab the model and labels first.
    
    Since your Colab "emulation" already has these downloaded from a previous step,
    you only need the below for the real Cloud Functions code.
    """

    # download_blob(BUCKET_NAME, 'tf-cloud-functions/test.jpeg',
    #              '/tmp/test.jpeg')
    # download_blob(BUCKET_NAME, 'tf-cloud-functions/frozen_inference_graph_image_tensor.pb',
    #              '/tmp/frozen_inference_graph_image_tensor.pb')
    # download_blob(BUCKET_NAME, 'tf-cloud-functions/label_map.pbtxt',
    #              '/tmp/label_map.pbtxt')

    results = []

    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')

    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            image = Image.open(PATH_TO_IMAGE)
            image_np = load_image_into_numpy_array(image)
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name(
                'num_detections:0')

            output_dict = run_inference_for_single_image(
                image_np, detection_graph)

            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})

            boxes = output_dict['detection_boxes']
            max_boxes_to_draw = boxes.shape[0]
            scores = output_dict['detection_scores']

            # Set your scoring threshold here (.7 = 70%)
            min_score_thresh = .7

            for i in range(min(max_boxes_to_draw, boxes.shape[0])):
                if scores is None or scores[i] > min_score_thresh:
                    _class_int = int(classes[0][i] - 1)
                    print('Class:', model_classes[_class_int])
                    print('Score:', scores[i])
                    print('Box:', boxes[i])

                    _result = {}
                    _result['class'] = model_classes[_class_int]
                    _result["score"] = json.dumps(str(scores[i]))
                    _result["box"] = json.dumps(str(boxes[i]))
                    results.append(_result)

    # Return JSON as per any normal HTTP response
    stringified_response = json.dumps(results)
    print(stringified_response)

    topic = TOPIC_PATH

    data = stringified_response
    data = data.encode("utf-8")

    encoded_file_name = file_name
    encoded_file_name = file_name.encode("utf-8")

    return stringified_response

In [0]:
# Explicitly run the handler() function in Colab
# No need for this in an actual Cloud Functions environment!
handler(None);